# Import Modules

In [ ]:
import importlib
import os
import sys

import joblib
import numpy as np
import pandas as pd
import polars as pl

In [ ]:
os.chdir("../")
sys.path.insert(0, os.getcwd())

In [ ]:
from morai.experience import tables
from morai.utils import custom_logger, helpers

In [ ]:
logger = custom_logger.setup_logging(__name__)

In [175]:
# update log level if wanting more logging
custom_logger.set_log_level("INFO")

In [6]:
pd.options.display.float_format = "{:,.2f}".format

# Load Model

In [ ]:
model_name = "glm"
GLM.model = joblib.load(f"files/models/{model_name}.joblib")
logger.info(f"loaded model '{model_name}'")

# Create Table

## Create Grid

In [ ]:
grid = tables.create_grid(mapping=mapping)
grid = grid.drop(columns=["vals", "capped_duration", "constant"])
grid = tables.remove_duplicates(grid, suppress_log=True)

durations = pd.DataFrame({"duration": range(1, 122)})
durations["key"] = 1
grid["key"] = 1
grid = pd.merge(grid, durations, on="key").drop(columns="key")
grid["duration"] = grid["duration"].astype(int)
grid["capped_duration"] = grid["duration"].clip(upper=26).astype(int)

grid = tables.check_aa_ia_dur_cols(grid)

## Predict

In [ ]:
pred = models.generate_table(
    model=GLM.model,
    mapping=mapping,
    feature_dict=preprocess_dict["feature_dict"],
    params=preprocess_dict["params"],
    grid=grid,
)
pred = tables.check_aa_ia_dur_cols(pred)

## Output

In [ ]:
tables.output_table(pred, "glm.csv")